## Install the requirements

In [1]:
!pip install -q llama-index==0.12.12 openai==1.59.6 tiktoken==0.8.0 llama-index-readers-web==0.3.4 firecrawl-py==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### SET THE ENVIRONMENT VARIABLES

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
FIRECRAWL_API_KEY = "<FIRECRAWL_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')
# os.environ["FIRECRAWL_API_KEY"] = userdata.get('FIRECRAWL_API_KEY')

# FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY')

# SCRAPE WITH FIRECRAWL

## IMPORT THE FIRECRAWL WEBREADER

Firecrawl allows you to turn entire websites into LLM-ready markdown

Get the API key here
https://www.firecrawl.dev/app/api-keys

In [3]:
from llama_index.readers.web import FireCrawlWebReader

In [4]:

# using firecrawl to crawl a website
firecrawl_reader = FireCrawlWebReader(
    api_key=FIRECRAWL_API_KEY,  # Replace with your actual API key from https://www.firecrawl.dev/
    mode="scrape",
)

# Load documents from a single page URL
documents = firecrawl_reader.load_data(url="https://towardsai.net/")

In [5]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [6]:
res = query_engine.query("What is towards AI aim?")

print(res.response)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['sourceURL'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("-_"*20)

Towards AI aims to be the world's leading artificial intelligence (AI) and technology publication, catering to thought-leaders and decision-makers globally by sharing high-quality publications, news, articles, and stories on AI and technology-related topics.
-----------------
Node ID	 67dc199c-12c8-4207-b09b-c4d77d01b235
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8477040848763497
Description	 ['Towards AI is an online publication, which focuses on sharing high-quality publications, news, articles, and stories on AI and technology related topics.', 'Our thoughts and highlights from the latest AI news, models and papers. Read by over 120,000 AI practitioners and students. Click to read Towards AI Newsletter, a Substack publication.']
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 1fa123a1-adfe-457a-b546-32b35b741318
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8463061955785798
Description	 ['Towards AI is an online publication, which focuses on sharing high-quali

# CRAWL A WEBSITE

## Load The CSV

CSV contains the list of tools and url of the page which we use to get information about the tool.

In [19]:
import requests
import csv

# Google Sheets file URL (CSV export link)
url = 'https://docs.google.com/spreadsheets/d/1gHB-aQJGt9Nl3cyOP2GorAkBI_Us2AqkYnfqrmejStc/export?format=csv'

# Send a GET request to fetch the CSV file
response = requests.get(url)

response_list = []
# Check if the request was successful
if response.status_code == 200:
    # Decode the content to a string
    content = response.content.decode('utf-8')

    # Use the csv.DictReader to read the content as a dictionary
    csv_reader = csv.DictReader(content.splitlines(), delimiter=',')
    response_list = [row for row in csv_reader]
else:
    print(f"Failed to retrieve the file: {response.status_code}")


In [20]:
import random

start_index = random.randint(0, len(response_list) - 3)
website_list = response_list[start_index:start_index+10] # Crawling 10 websites only.

In [21]:
import pprint
print("CSV data")
pprint.pprint(website_list)

CSV data
[{'': '',
  'Category': 'Distributed Training',
  'Company': 'Nvidia',
  'Description': 'Multi-GPU and multi-node collective communication primitives',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'NVIDIA NCCL',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://developer.nvidia.com/nccl'},
 {'': '',
  'Category': 'Model Optimization',
  'Company': '',
  'Description': 'High-performance deep learning inference optimizer and '
                 'runtime',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'NVIDIA TensorRT',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://developer.nvidia.com/tensorrt'},
 {'': '',
  'Category': 'Computer Vision',
  'Company': '',
  'Description': 'Computer vision and machine learning software library',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'OpenCV',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://opencv.org/'},
 {'': '',
  'Category': 'Computer Vision',
  'Company':

## Initialize the Firecrawl

In [22]:
import os
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

In [23]:
import time

# Crawl websites and handle responses
url_response = {}
crawl_per_min = 1  # Max crawl per minute

# Track crawls
crawled_websites = 0
scraped_pages = 0

for i, website_dict in enumerate(website_list):
    url = website_dict.get('URL')
    print(f"Crawling: {url}")

    try:
        response = app.crawl_url(
            url,
            params={
                'limit': 5,  # Limit pages to scrape per site.
                'scrapeOptions': {'formats': ['markdown', 'html']}
            }
        )
        crawled_websites += 1

    except Exception as exc:
        print(f"Failed to fetch {url} -> {exc}")
        continue

    # Store the scraped data and associated info in the response dict
    url_response[url] = {
        "scraped_data": response.get("data"),
        "csv_data": website_dict
    }

    # Pause to comply with crawl per minute limit for free version its 1 crawl per minute
    if i!=len(website_list) and (i + 1) % crawl_per_min == 0:
        print("Pausing for 1 minute to comply with crawl limit...")
        time.sleep(60)  # Pause for 1 minute after every crawl


Crawling: https://developer.nvidia.com/nccl
Pausing for 1 minute to comply with crawl limit...
Crawling: https://developer.nvidia.com/tensorrt
Pausing for 1 minute to comply with crawl limit...
Crawling: https://opencv.org/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://blog.roboflow.com/what-is-openpose/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pandas-ai.com/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pypi.org/project/pandas-profiling/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pandera.readthedocs.io/en/stable/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pypi.org/project/panel/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pypi.org/project/Pattern/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pettingzoo.farama.org/index.html
Pausing for 1 minute to comply with crawl limit...


## Create  llamaindex documents from the scraped content

In [24]:
from llama_index.core import Document
documents = []

for _, scraped_content in url_response.items():
    csv_data = scraped_content.get("csv_data")
    scraped_results = scraped_content.get("scraped_data")

    for scraped_site_dict in scraped_results:
        for result in scraped_results:
            markdown_content = result.get("markdown")
            title = result.get("metadata").get("title")
            url = result.get("metadata").get("sourceURL")
            documents.append(
                Document(
                    text=markdown_content,
                    metadata={
                        "title": title,
                        "url": url,
                        "description": csv_data.get("Description"),
                        "category": csv_data.get("Category")
                    }
                )
            )


# Create The RAG Pipeline.

In [25]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

llm = OpenAI(model="gpt-4o-mini")
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [26]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [27]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [28]:
from IPython.display import Markdown, display
def display_response(response):
    display(Markdown(f"<b>{response}</b>"))

In [29]:
query = "What is Real-time multi-person keypoint detection" # Enter your query here, it should be relevant to the crawled websites
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

<b>Real-time multi-person keypoint detection refers to the process of identifying and tracking specific points on the human body, such as joints and limbs, in a video stream or image. This technology enables the detection of multiple individuals simultaneously, allowing for applications in various fields such as healthcare, entertainment, and sports. By analyzing the positions of these keypoints, systems can interpret human movements and poses in real-time, facilitating interactive experiences and providing valuable insights for monitoring and analysis.</b>

-----------------
Node ID	 b61fc79e-969d-429c-804e-f888272fbd8d
Title	 What is OpenPose? A Guide for Beginners.
URL	 https://blog.roboflow.com/what-is-openpose/
Score	 0.5726111768661022
Description	 Real-time multi-person keypoint detection library
Category	 Computer Vision
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 578b8dc1-131d-4acb-b9a1-268f02925da9
Title	 What is OpenPose? A Guide for Beginners.
URL	 https://blog.roboflow.com/what-is-openpose/
Score	 0.5217128971469592
Description	 Real-time multi-person keypoint detection library
Category	 Computer Vision
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_


In [30]:
# Querying not relevant to the crawled websites.

query = "What is qdrant?"
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

<b>The provided information does not include any details about Qdrant. It focuses on a data validation framework called Pandera, which is used for validating data in dataframe-like objects. For information about Qdrant, you may need to consult other sources.</b>

-----------------
Node ID	 b9560b11-f0ca-41c5-95ac-a1afe5ad6c96
Title	 pandera documentation
URL	 https://pandera.readthedocs.io/en/stable/
Score	 0.1993697291056966
Description	 Statistical data validation for pandas
Category	 Data Validation
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
